In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:95% !important;}</style>"))
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [88]:
#from seq_generator.query_generator import *
#from seq_generator.mismatch_generator import *
from encoder_decoder.seq_encoder_decoder import *
#import encoder_decoder.seq_encoder_decoder
from seq_generator.data_generator import *
from job_script.cas_offinder_job import *

In [4]:
import numpy as np

In [5]:
job_name = 'mis_save_test'
path = '.'

In [6]:
tmp_data = generate_8_nC2_data(
    num_set = 3,
    threshold = 7,
    log = True,
    job_name = job_name,
    save_path = path,
)

In [13]:
tmp_data.ontarget_data

array(['CCCAAAAAAGATTCATATGT', 'ACATTACAGACTTGTGCAGG',
       'ATTATAAGGAGGGGTTCCAT', 'TTTATGTCCCACCGCGTCCC',
       'GGCCGGTATATACCAGCAAA', 'GGACGGGTGAATAGCGGTAC',
       'CGGCCGTCGCCCCCAAGGAT', 'ACGGTCACCGCAGGGGGGGT',
       'CTTTTTTTATTTTTGTTTTA', 'GTAGAATTTCATGTAGGCTC',
       'CCGTAAAACCTGCGATATTA', 'TTTATTTCACCGCGCCGCAA',
       'GCTCGGGCTAACCAATATTT', 'GATTGGCATTATTACGGCTC',
       'CATGGGACGAGCATTGATAA', 'CCCCCGGGCCGGCGCGCTCA',
       'TATGTATTACTGTTTTTTCA', 'TCAGATAAAAATTTGGCGCG',
       'AATTACTGGGATGGGAAAAT', 'TTTCTACGCGGGAGTGTGCG',
       'CTACGGCAAAGTAATCTTTG', 'CGAAAGGACTACAACCGGAC',
       'GCTCCACGCACGAATATAAA', 'GGGCCTGCCCCCTGCTCGCC'], dtype='<U20')

In [10]:
tmp_data.mis_1_data.shape

(24, 60)

In [16]:
tmp_data.mis_2_data.shape

(24, 570)

In [18]:
tmp_data.mis_2_dis_data.shape

(24, 80)

In [ ]:
#run_cas_offinder(
#    file_path = f"{path}/{job_name}/run_{job_name}_script.sh",
#    log = True
#)

In [ ]:
tmp_df = analysis_cas_offinder_result(
    output_file_path = f"{path}/{job_name}/{job_name}_output.txt",
    query_text_path = f"{path}/{job_name}/query_{job_name}.txt",
)

In [ ]:
tmp_df

In [ ]:
def query_seq_generate(job_name, path='./', method=generate_8_nC2_data):
    job_path = f"{path}/{job_name}"
    cas_offinder_script_path = f"{job_path}/run_{job_name}_script.sh"
    output_file_path = f"{job_path}/{job_name}_output.txt"
    query_file_path = f"{job_path}/{job_name}_query.txt"
    data = method(
        num_set = 3,
        threshold = 7,
        log = True,
        job_name = job_name,
        save_path = path,
    )
    make_cas_offinder_script(
        query_seq_list = data.query_data,
        job_name = job_name,
        job_path = job_path,
    )
    run_cas_offinder(
        file_path = cas_offinder_script_path,
        log = True,
    )
    result_df = analysis_cas_offinder_result(
        output_file_path = output_file_path,
        query_text_path = query_file_path,
    )
    return result_df

In [ ]:
query_seq_generate(
    job_name = "pipeline_test",
    path = './test',
    method = generate_8_nC2_data,
)

In [ ]:
tmp_data = generate_8_nC2_data(
    num_set = 1,
    threshold = 7,
    log = True,
    job_name = 'test',
    save_path = '.'
)

In [ ]:
tmp_query = tmp_data.query_data
tmp_query

In [ ]:
make_cas_offinder_script(
    query_seq_list = tmp_query,
    job_name = 'test_job',
    job_path = f'./test',
)

In [ ]:
seq_list = tmp_query
batch_type_code_seq_list = []
for _seq in seq_list:
    _type_code = query_type_code_encoder.seq_type_code_encoder(_seq)
    batch_type_code_seq_list.append((_type_code,_seq))

In [ ]:
import pandas as pd

In [ ]:
analysis_cas_offinder_result(
    output_file_path='./test/test_job_out.txt',
    query_text_path='./test/test_query.txt',
    save_csv=True,
)

In [ ]:
command = ["bash",'../test/run_test_job_script.sh']
cas_offinder_run = subprocess.Popen(
    command,
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
)

In [ ]:
output, log = cas_offinder_run.communicate()

In [ ]:
print(output)

In [ ]:
print(log)

In [79]:
_seq_1 = 'ACGTAACCGGTT'
_seq_1_one_hot = one_hot_encoder.seq_encoder(_seq_1)
_seq_2 = 'ACGCAAACGGTT'
_seq_2_one_hot = one_hot_encoder.seq_encoder(_seq_2)

In [80]:
_seq_1_prime = mismatch_calculator.seq_prime_encoder(_seq_1,mismatch_calculator.prime_main_encoder_dict)
_seq_2_prime= mismatch_calculator.seq_prime_encoder(_seq_1,mismatch_calculator.prime_sub_encoder_dict)


In [89]:
mismatch_info = mismatch_calculator.seq_mismatch_calculator(_seq_1,_seq_2)
mismatch_info

[None, None, None, 'T>C', None, None, 'C>A', None, None, None, None, None]

In [90]:
mis_info_list = []
for index, _info in enumerate(mismatch_info):
    if _info:
        _tmp_info = f"{index}:{_info}"
        mis_info_list.append(_tmp_info)

In [91]:
mis_info_list

['3:T>C', '6:C>A']

In [96]:
','.join(mis_info_list)

'3:T>C,6:C>A'

In [43]:
seq_one_hot = seq_one_hot
seq_tmp = []
for letter_one_hot in seq_one_hot:
    one_hot_pos_list = np.array(letter_one_hot, dtype=np.int8)
    one_hot_pos_code = ''.join(map(str,one_hot_pos_list))
    seq_tmp.append(one_hot_pos_code)

In [44]:
seq_tmp

['0000',
 '0000',
 '0000',
 '0-101',
 '0000',
 '0000',
 '0000',
 '0000',
 '0000',
 '0000',
 '0000',
 '0000']

In [48]:
list_1 = [1,2,3,5]
list_2 = [7,11,13,17]
for l1 in list_1:
    for l2 in list_2:
        print(l1*l2)

7
11
13
17
14
22
26
34
21
33
39
51
35
55
65
85


In [51]:
np.int8(-127)

-127